# Import libraries

In [2]:
pip install ortools

Note: you may need to restart the kernel to use updated packages.


In [3]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import data

In [4]:
repo_url = 'https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'
full_data = pd.read_csv(repo_url + 'full_data.csv')
distance_matrix = pd.read_csv(repo_url + 'distance_matrix.csv',header=None)
vehicle_matrix = pd.read_excel(repo_url + 'vhc_matrix_city_excluded.xlsx')

In [5]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,229,230,231,232,233,234,235,236,237,238
0,inf,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,4.4526,...,68.2641,68.5917,68.6752,69.3386,69.9441,72.1044,72.2981,74.0778,77.0503,89.0705
1,2.6786,inf,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,5.7512,...,66.3784,70.7477,70.8312,71.4946,68.0584,74.2604,74.4541,76.2338,79.2062,91.2265
2,2.8743,3.9804,inf,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,2.2389,...,68.1229,68.7464,68.8299,69.4933,69.8029,72.2591,72.4528,74.2325,77.2050,89.2252
3,4.0302,3.0074,4.1825,inf,5.8087,7.6315,6.1501,3.6034,7.4234,5.9533,...,64.2690,70.9498,71.0333,71.6967,65.9490,74.4625,74.6562,76.4359,79.4084,91.4286
4,4.1759,5.6066,2.3205,5.8087,inf,3.2004,8.0162,8.1495,8.3665,2.7008,...,67.9534,68.5994,68.6829,69.3463,69.6334,72.1121,72.3058,74.0855,77.0580,89.0782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,72.1044,74.2604,72.2591,74.4625,72.1121,71.5321,75.9480,76.1065,76.2023,73.6675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,72.2981,74.4541,72.4528,74.6562,72.3058,71.7258,76.1417,76.3002,76.3960,73.8612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,74.0778,76.2338,74.2325,76.4359,74.0855,73.5055,77.9214,78.0799,78.1757,75.6409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,77.0503,79.2062,77.2050,79.4084,77.0580,76.4779,80.8939,81.0524,81.1482,78.6133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
demands_wed =  [0] + full_data['wed_tickets'].values.tolist()
demands_thu =  [0] + full_data['thu_tickets'].values.tolist()
demands_fri =  [0] + full_data['fri_tickets'].values.tolist()
demands_sat =  [0] + full_data['sat_tickets'].values.tolist()

In [7]:
venoge = [float('inf')]
venoge.extend(full_data['distance_to_venoge_km'].values.tolist())
df_column = pd.DataFrame(venoge)
df_row = pd.DataFrame(full_data['distance_to_venoge_km'].values.tolist()).T

In [8]:
distance_matrix = pd.concat([df_row,distance_matrix],ignore_index=True)
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,229,230,231,232,233,234,235,236,237,238
0,0.7400,2.7200,3.5100,3.9800,4.1200,4.3200,4.7200,4.8500,5.0400,5.0800,...,68.2100,68.5400,68.6200,69.2800,69.8900,72.0500,72.2400,74.0200,77.0000,89.0200
1,inf,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,4.4526,...,68.2641,68.5917,68.6752,69.3386,69.9441,72.1044,72.2981,74.0778,77.0503,89.0705
2,2.6786,inf,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,5.7512,...,66.3784,70.7477,70.8312,71.4946,68.0584,74.2604,74.4541,76.2338,79.2062,91.2265
3,2.8743,3.9804,inf,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,2.2389,...,68.1229,68.7464,68.8299,69.4933,69.8029,72.2591,72.4528,74.2325,77.2050,89.2252
4,4.0302,3.0074,4.1825,inf,5.8087,7.6315,6.1501,3.6034,7.4234,5.9533,...,64.2690,70.9498,71.0333,71.6967,65.9490,74.4625,74.6562,76.4359,79.4084,91.4286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,72.1044,74.2604,72.2591,74.4625,72.1121,71.5321,75.9480,76.1065,76.2023,73.6675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,72.2981,74.4541,72.4528,74.6562,72.3058,71.7258,76.1417,76.3002,76.3960,73.8612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,74.0778,76.2338,74.2325,76.4359,74.0855,73.5055,77.9214,78.0799,78.1757,75.6409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,77.0503,79.2062,77.2050,79.4084,77.0580,76.4779,80.8939,81.0524,81.1482,78.6133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
distance_matrix.columns = range(1, len(distance_matrix.columns) + 1)
distance_matrix

,1,2,3,4,5,6,7,8,9,10,...,230,231,232,233,234,235,236,237,238,239
0,0.7400,2.7200,3.5100,3.9800,4.1200,4.3200,4.7200,4.8500,5.0400,5.0800,...,68.2100,68.5400,68.6200,69.2800,69.8900,72.0500,72.2400,74.0200,77.0000,89.0200
1,inf,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,4.4526,...,68.2641,68.5917,68.6752,69.3386,69.9441,72.1044,72.2981,74.0778,77.0503,89.0705
2,2.6786,inf,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,5.7512,...,66.3784,70.7477,70.8312,71.4946,68.0584,74.2604,74.4541,76.2338,79.2062,91.2265
3,2.8743,3.9804,inf,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,2.2389,...,68.1229,68.7464,68.8299,69.4933,69.8029,72.2591,72.4528,74.2325,77.2050,89.2252
4,4.0302,3.0074,4.1825,inf,5.8087,7.6315,6.1501,3.6034,7.4234,5.9533,...,64.2690,70.9498,71.0333,71.6967,65.9490,74.4625,74.6562,76.4359,79.4084,91.4286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,72.1044,74.2604,72.2591,74.4625,72.1121,71.5321,75.9480,76.1065,76.2023,73.6675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,72.2981,74.4541,72.4528,74.6562,72.3058,71.7258,76.1417,76.3002,76.3960,73.8612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,74.0778,76.2338,74.2325,76.4359,74.0855,73.5055,77.9214,78.0799,78.1757,75.6409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,77.0503,79.2062,77.2050,79.4084,77.0580,76.4779,80.8939,81.0524,81.1482,78.6133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
distance_matrix.insert(0,0,df_column)
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,inf,0.7400,2.7200,3.5100,3.9800,4.1200,4.3200,4.7200,4.8500,5.0400,...,68.2100,68.5400,68.6200,69.2800,69.8900,72.0500,72.2400,74.0200,77.0000,89.0200
1,0.74,inf,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,...,68.2641,68.5917,68.6752,69.3386,69.9441,72.1044,72.2981,74.0778,77.0503,89.0705
2,2.72,2.6786,inf,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,...,66.3784,70.7477,70.8312,71.4946,68.0584,74.2604,74.4541,76.2338,79.2062,91.2265
3,3.51,2.8743,3.9804,inf,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,...,68.1229,68.7464,68.8299,69.4933,69.8029,72.2591,72.4528,74.2325,77.2050,89.2252
4,3.98,4.0302,3.0074,4.1825,inf,5.8087,7.6315,6.1501,3.6034,7.4234,...,64.2690,70.9498,71.0333,71.6967,65.9490,74.4625,74.6562,76.4359,79.4084,91.4286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,72.05,72.1044,74.2604,72.2591,74.4625,72.1121,71.5321,75.9480,76.1065,76.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,72.24,72.2981,74.4541,72.4528,74.6562,72.3058,71.7258,76.1417,76.3002,76.3960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,74.02,74.0778,76.2338,74.2325,76.4359,74.0855,73.5055,77.9214,78.0799,78.1757,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,77.00,77.0503,79.2062,77.2050,79.4084,77.0580,76.4779,80.8939,81.0524,81.1482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
distance_matrix.replace(np.inf, 0, inplace=True)

In [12]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,0.00,0.7400,2.7200,3.5100,3.9800,4.1200,4.3200,4.7200,4.8500,5.0400,...,68.2100,68.5400,68.6200,69.2800,69.8900,72.0500,72.2400,74.0200,77.0000,89.0200
1,0.74,0.0000,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,...,68.2641,68.5917,68.6752,69.3386,69.9441,72.1044,72.2981,74.0778,77.0503,89.0705
2,2.72,2.6786,0.0000,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,...,66.3784,70.7477,70.8312,71.4946,68.0584,74.2604,74.4541,76.2338,79.2062,91.2265
3,3.51,2.8743,3.9804,0.0000,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,...,68.1229,68.7464,68.8299,69.4933,69.8029,72.2591,72.4528,74.2325,77.2050,89.2252
4,3.98,4.0302,3.0074,4.1825,0.0000,5.8087,7.6315,6.1501,3.6034,7.4234,...,64.2690,70.9498,71.0333,71.6967,65.9490,74.4625,74.6562,76.4359,79.4084,91.4286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,72.05,72.1044,74.2604,72.2591,74.4625,72.1121,71.5321,75.9480,76.1065,76.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,72.24,72.2981,74.4541,72.4528,74.6562,72.3058,71.7258,76.1417,76.3002,76.3960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,74.02,74.0778,76.2338,74.2325,76.4359,74.0855,73.5055,77.9214,78.0799,78.1757,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,77.00,77.0503,79.2062,77.2050,79.4084,77.0580,76.4779,80.8939,81.0524,81.1482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
subset_distance_matrix0=distance_matrix.iloc[:5,:5].astype(int).values.tolist()

In [14]:
subset_distance_matrix0

[[0, 0, 2, 3, 3],
 [0, 0, 2, 2, 4],
 [2, 2, 0, 3, 3],
 [3, 2, 3, 0, 4],
 [3, 4, 3, 4, 0]]

In [15]:
subset_demands0=demands_wed[:5]

In [16]:
subset_demands0

[0, 246.0, 237.0, 37.0, 74.0]

In [17]:
subset_demands1 = [int(i) for i in subset_demands0]
subset_demands1

[0, 246, 237, 37, 74]

In [18]:
distance_matrix = distance_matrix.values.tolist()

In [19]:
subset_demands = demands_wed[:26]

In [21]:
capacities=vehicle_matrix['Places'].values.tolist()

In [22]:
capacities

[16, 91, 50, 20, 127, 100]

In [23]:
capacities=capacities*25

# Draft model

In [24]:
# Filter out cities with zero demand
def reduce_problem(distance_matrix, demands):
    indices_to_keep = [i for i, d in enumerate(demands) if d > 0 or i == 0]  # keep depot too
    reduced_matrix = distance_matrix[np.ix_(indices_to_keep, indices_to_keep)]
    reduced_demands = [demands[i] for i in indices_to_keep]
    return reduced_matrix, reduced_demands, indices_to_keep


In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model(distance_matrix, demands,capacities):
    """Stores the data for the problem."""
    data = {}
    # Data multiplied by a factor of 10 to avoid non-integer numbers
    data['distance_matrix'] = distance_matrix
    data['demands'] = demands
    data['vehicle_capacities'] = [16, 91, 50, 20, 127, 500]
    data['num_vehicles'] = len(capacities)
    data['depot'] = 0
    return data

'''
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}km\n'.format(route_distance/10)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}km'.format(total_distance/10))
    print('Total load of all routes: {}'.format(total_load))
'''
def print_solution(data, manager, routing, solution):
    """Prints solution on console with clear output for active and unused vehicles."""
    total_distance = 0
    total_load = 0

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        route_load = 0
        route_distance = 0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            route_load += data['demands'][node_index]
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        end_node = manager.IndexToNode(index)
        route.append(end_node)

        # Detect unused vehicle (only [0, 0])
        if route == [0, 0]:
            print(f"Vehicle {vehicle_id}: unused")
            continue

        # Print full route for used vehicle
        print(f"Route for vehicle {vehicle_id}:")
        for node in route:
            print(f" {node} (Load: {data['demands'][node]})", end=" ->")
        print(f"\nTotal distance: {route_distance / 10:.2f} km")
        print(f"Total load: {route_load}\n")

        total_distance += route_distance
        total_load += route_load

    print(f"Total distance of all routes: {total_distance / 10:.2f} km")
    print(f"Total load of all routes: {total_load}")

def main():
    """Solve the CVRP problem."""

    ## TODO: Complete here
    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix0, subset_demands1, capacities)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')


main()



: 

In [1]:
subset_distance_matrix

NameError: name 'subset_distance_matrix' is not defined

In [ ]:


from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model(subset_distance_matrix, subset_demands, capacities):
    """Stores the data for the problem."""
    data = {}
    # Data multiplied by a factor of 10 to avoid non-integer numbers
    data['distance_matrix'] = subset_distance_matrix
    data['demands'] = subset_demands
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = 3
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}km\n'.format(route_distance/10)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}km'.format(total_distance/10))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Solve the CVRP problem."""

    ## TODO: Complete here
    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix=subset_distance_matrix, subset_demands=subset_demands, capacities=capacities)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        print_solution(data, manager, routing, solution)


main()

: 